# Traitement des donnees

In [11]:
import json
import os
import spacy
import re

# Charger le modèle français de spaCy
try:
    nlp = spacy.load("fr_core_news_sm")
except:
    import subprocess
    subprocess.run(["python", "-m", "spacy", "download", "fr_core_news_sm"])
    nlp = spacy.load("fr_core_news_sm")

# Charger le fichier product_links.json
def load_product_links(file_path):
    """
    Charger les données des liens produits depuis un fichier JSON.

    Args:
        file_path (str): Chemin du fichier JSON contenant les liens produits.

    Returns:
        dict: Données des liens produits.
    """
    print(f"Chargement des données produits depuis {file_path}...")
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
    print(f"{len(data)} catégories chargées avec succès.")
    return data

# Charger les fichiers comments
def load_comments_files(comments_folder):
    """
    Charger les commentaires produits depuis plusieurs fichiers JSON.

    Args:
        comments_folder (str): Chemin du dossier contenant les fichiers de commentaires.

    Returns:
        dict: Données des commentaires combinées pour tous les produits.
    """
    print(f"Chargement des fichiers de commentaires depuis {comments_folder}...")
    all_comments = {}
    for file_name in os.listdir(comments_folder):
        if file_name.endswith(".json"):
            print(f"  Chargement de {file_name}...")
            with open(os.path.join(comments_folder, file_name), "r", encoding="utf-8") as file:
                comments_data = json.load(file)
                all_comments.update(comments_data)
    print(f"{len(all_comments)} produits avec commentaires chargés avec succès.")
    return all_comments

# Nettoyer et traiter un texte avec spaCy, en prenant en compte la négation
def process_text_with_spacy(text, stop_words):
    """
    Nettoyer et traiter un texte pour extraire les mots-clés importants tout en prenant en compte la négation.

    Args:
        text (str): Texte brut à traiter.
        stop_words (set): Ensemble des stop-words personnalisés.

    Returns:
        str: Texte traité et nettoyé.
    """
    if not text:
        return ""
    doc = nlp(text.lower())  # Convertir en minuscules et analyser avec spaCy
    tokens = []
    negation_active = False  # Indicateur de négation

    for token in doc:
        if token.is_stop or token.text in stop_words or not token.is_alpha:
            continue  # Ignorer les stop-words et les tokens non alphabétiques

        if token.text in {"ne", "pas", "jamais", "rien", "aucun", "ni"}:
            negation_active = True  # Activer la négation
            tokens.append(token.text)  # Inclure le mot de négation
        elif negation_active:
            # Ajouter le mot négatif combiné au token suivant
            tokens.append(f"{token.lemma_}_{tokens.pop()}")
            negation_active = False  # Désactiver la négation après association
        else:
            tokens.append(token.lemma_)  # Ajouter le lemme normalisé

    return " ".join(tokens)

# Convertir les champs average_rating et total_reviews
def convert_fields(product_data):
    """
    Convertir les champs average_rating en float et total_reviews en int.

    Args:
        product_data (dict): Données d'un produit contenant ces champs.
    """
    try:
        product_data["average_rating"] = float(product_data["average_rating"])
    except (ValueError, TypeError):
        product_data["average_rating"] = None

    try:
        match = re.search(r"\d+", product_data["total_reviews"])
        product_data["total_reviews"] = int(match.group()) if match else None
    except (ValueError, TypeError):
        product_data["total_reviews"] = None

# Fusionner et nettoyer les fichiers
def merge_and_clean_data(product_links, comments):
    """
    Fusionner les données des produits avec leurs commentaires et nettoyer les textes.

    Args:
        product_links (dict): Données des liens produits.
        comments (dict): Données des commentaires des produits.

    Returns:
        list: Données fusionnées et nettoyées.
    """
    print("Début du processus de fusion et de nettoyage des données...")
    stop_words = {"très", "bon", "bien", "moyen", "avis", "produit",
                  "super", "aimer", "article", "mal", "goût", "hyper", "m", "exemple", "image", "prendre", "fois",
                  "j", "porter", "recommande", "conseiller", "stricte", "rigoureux", "recommander", "voir", "cas", "hum",
                  "temps", "top", "remercier", "savoir", "beaucoup", "bon", "vrai", "decu", "conseille", "espérer",
                  "colis", "venir"}  # Stop-words personnalisés

    merged_cleaned_data = []

    for category, products in product_links.items():
        print(f"Traitement de la catégorie : {category}...")
        for product in products:
            product_link = product["link"]
            print(f"  Traitement du produit : {product_link}...")
            product_data = {
                "link": product_link,
                "name": product["name"],
                "price": product["price"],
                "reviews": product["reviews"],
                "average_rating": comments.get(product_link, {}).get("average_rating"),
                "total_reviews": comments.get(product_link, {}).get("total_reviews"),
                "comments": comments.get(product_link, {}).get("comments", [])
            }

            # Convertir les champs average_rating et total_reviews
            convert_fields(product_data)

            # Nettoyage des commentaires
            for comment in product_data["comments"]:
                # Traitement du title
                comment_title = comment.get("title", "")
                comment["processed_title"] = process_text_with_spacy(comment_title, stop_words)

                # Traitement du content
                comment_text = comment.get("content", "")
                comment["processed_content"] = process_text_with_spacy(comment_text, stop_words)

            merged_cleaned_data.append(product_data)

    print("Fusion et nettoyage des données terminés.")
    return merged_cleaned_data

# Sauvegarder les données fusionnées et nettoyées
def save_final_data(data, output_file):
    """
    Sauvegarder les données finales dans un fichier JSON.

    Args:
        data (list): Données fusionnées et nettoyées.
        output_file (str): Chemin du fichier de sortie.
    """
    print(f"Sauvegarde des données finales dans {output_file}...")
    with open(output_file, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
    print("Données sauvegardées avec succès.")

# Chemins des fichiers
product_links_file = r"C:\TOUS_MES_FICHIERS\ProjetPython\TravauxTechniques\1scraping\datasets\products\product_links.json"
comments_folder = r"C:\TOUS_MES_FICHIERS\ProjetPython\TravauxTechniques\1scraping\datasets\comments"
output_file = r"C:\TOUS_MES_FICHIERS\ProjetPython\TravauxTechniques\2traitement\datasets\data_fusion_cleaned.json"

# Processus combiné
print("Chargement des données...")
product_links = load_product_links(product_links_file)
comments = load_comments_files(comments_folder)
final_data = merge_and_clean_data(product_links, comments)
save_final_data(final_data, output_file)


Chargement des données...
Chargement des données produits depuis C:\TOUS_MES_FICHIERS\ProjetPython\TravauxTechniques\1scraping\datasets\products\product_links.json...
12 catégories chargées avec succès.
Chargement des fichiers de commentaires depuis C:\TOUS_MES_FICHIERS\ProjetPython\TravauxTechniques\1scraping\datasets\comments...
  Chargement de comments_accessoires-hommes-mode.json...
  Chargement de comments_baskets-de-ville.json...
  Chargement de comments_bermudas-homme.json...
  Chargement de comments_boots-hommes.json...
  Chargement de comments_derbies.json...
  Chargement de comments_jeans-homme.json...
  Chargement de comments_mocassins-chaussures-bateau-hommes.json...
  Chargement de comments_polos.json...
  Chargement de comments_sandales-tongs-hommes.json...
  Chargement de comments_sous-vetements.json...
  Chargement de comments_tee-shirts-homme.json...
  Chargement de comments_vetements-chemises-occasionnelles-bouton-vers-bas.json...
2423 produits avec commentaires charg